<a href="https://colab.research.google.com/github/yihui-Xiong/Fa23-CLS-0161-01-Intro-Dig-Hum/blob/main/Another_copy_of_chronicling_america_api(complete).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# imports
import requests
import json
import math
import pandas as pd
import spacy

In [ ]:
# initial search
url = 'https://chroniclingamerica.loc.gov/search/pages/results/?state=Massachusetts&date1=1770&date2=1865&proxtext=prohibition&x=20&y=8&dateFilterType=yearRange&rows=20&searchType=basic&format=json'
response = requests.get(url)
raw = response.text
results = json.loads(raw)

In [ ]:
results.keys()

In [ ]:
# explore items
print(type(results['items']))

In [ ]:
print(results['items'][0])

In [ ]:
print('totalItems:', results['totalItems'])
print('endIndex:', results['endIndex'])
print('startIndex:', results['startIndex'])
print('itemsPerPage:', results['itemsPerPage'])
print('Length and type of items:', len(results['items']), type(results['items']))

In [ ]:
# find total amount of pages
total_pages = math.ceil(results['totalItems'] / results['itemsPerPage'])
print(total_pages)

In [ ]:
# create empty list for data
data = []

In [ ]:
# set search parameters
start_date = '1861'
end_date = '1865'
search_term = 'Civil War'
state = 'Massachusetts'

In [ ]:
# loop through search results and collect data
for i in range(1, 101):
    url = (f'https://chroniclingamerica.loc.gov/search/pages/results/?state={state}&date1={start_date}'
           f'&date2={end_date}&proxtext={search_term}&x=16&y=8&dateFilterType=yearRange&rows=20'
           f'&searchType=basic&format=json&page={i}')  # f-string
    response = requests.get(url)
    raw = response.text
    print(f'page {i} status code:', response.status_code)  # checking for errors
    results = json.loads(raw)
    items_ = results['items']
    for item_ in items_:
        row_data = {}
        try:
          row_data['title'] = item_['title_normal']
        except:
          row_data['city'] = "none"
        try:
          row_data['city'] = item_['city']
        except:
          row_data['city'] = "none"
        try:
          row_data['date'] = item_['date']
        except:
          row_data['date'] = "none"
        try:
          row_data['raw_text'] = item_['ocr_eng']
        except:
          row_data['raw_text'] = 'none'
    data.append(row_data)

In [ ]:
# put data into DataFrame
df = pd.DataFrame.from_dict(data)

In [ ]:
df.head()

In [ ]:
# convert date column from string to date-time object
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.head()

In [ ]:
# sort by date
df = df.sort_values(by='date')

In [ ]:
df.head()

In [ ]:
# write fuction to process text
# load nlp model
nlp = spacy.load("en_core_web_sm")
nlp.disable_pipes('ner', 'parser')  # these are unnecessary for the task at hand

def process_text(text):
    """Remove new line characters and lemmatize text. Returns string of lemmas"""
    text = text.replace('\n', ' ')
    doc = nlp(text)
    tokens = [token for token in doc]
    no_stops = [token for token in tokens if not token.is_stop]
    no_punct = [token for token in no_stops if token.is_alpha]
    lemmas = [token.lemma_ for token in no_punct]
    lemmas_lower = [lemma.lower() for lemma in lemmas]
    lemmas_string = ' '.join(lemmas_lower)
    return lemmas_string

In [ ]:
# apply process_text function
df['lemmas'] = df['raw_text'].apply(process_text)

In [ ]:
# save to csv
df.to_csv(f'/content/{search_term}{start_date}-{end_date}.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')